<a href="https://colab.research.google.com/github/HuyenNguyenHelen/LING-5412/blob/main/Assignment2_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing libraries that will be used 
import numpy as np
import tarfile
import glob
import re
import pandas as pd
#from nltk.corpus import stopwords
import nltk
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Untar the dataset
my_tar = tarfile.open('/content/review_polarity.tar.gz')
my_tar.extractall('/content/') 
my_tar.close()


In [ ]:
# Exploring the data sizes

paths_pos = glob.glob('/content/txt_sentoken/pos/*.txt')
paths_neg = glob.glob('/content/txt_sentoken/neg/*.txt')
pos_neg_paths = paths_pos + paths_neg

n_pos = len(paths_pos)
n_neg = len(paths_neg)

print('the number of positive instances: {} \nthe number of positive instances: {}'.format(n_pos, n_neg))

the number of positive instances: 1000 
the number of positive instances: 1000


In [ ]:
# Exploring the words in the dataset

from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

def tokenizer (doc):
  #doc = doc.lower() # Lowercase documents
  return re.split("\W+", doc)   # return a list of tokens without punctuations

# def BOW (doc):
#   bow = set()
#   for token in tokenizer (doc):
#     bow.add(token)
#   return list(bow)

#def word_counter (doc):

def stopword_remover (bow):
  filtered_bow = [w for w in bow if not w.lower() in stopwords]
  return filtered_bow

def top_freq_w (freq_dic, top_n):
  sorted_dic = {k:v for k, v in sorted(freq_dic.items(), key = lambda item: item[1], reverse=True)[:top_n]}
  return sorted_dic


word_freq = {}
for path in pos_neg_paths:
  fo = open(path)
  doc = fo.read()
  for token in tokenizer (doc):
    word_freq[token] = word_freq.get(token,0)+1

top_100_w = top_freq_w(word_freq, 100) 

print('the number of unique words in the dataset: ', len(word_freq.keys()))
print ('top 100 most frequent words:\n', top_100_w )
print('\nthe number of words in the top 100 which are stopwords: ', len([w for w in top_100_w.keys() if w in stopwords]))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
the number of unique words in the dataset:  39697
top 100 most frequent words:
 {'the': 76529, 'a': 38106, 'and': 35576, 'of': 34123, 'to': 31937, 'is': 25195, 'in': 21822, 's': 18513, 'it': 16107, 'that': 15924, 'as': 11378, 'with': 10792, 'for': 9961, 'his': 9587, 'this': 9578, 'film': 9517, 'i': 8889, 'he': 8864, 'but': 8634, 'on': 7385, 'are': 6949, 't': 6410, 'by': 6261, 'be': 6174, 'one': 5852, 'movie': 5771, 'an': 5744, 'who': 5692, 'not': 5577, 'you': 5316, 'from': 4999, 'at': 4986, 'was': 4940, 'have': 4901, 'they': 4825, 'has': 4719, 'her': 4522, 'all': 4373, 'there': 3770, 'like': 3690, 'so': 3683, 'out': 3637, 'about': 3523, 'up': 3405, 'more': 3347, 'what': 3322, 'when': 3258, 'which': 3161, 'or': 3148, 'she': 3141, 'their': 3122, 'some': 2985, 'just': 2905, 'can': 2882, 'if': 2799, 'we': 2775, 'him': 2633, 'into': 2623, 'even': 2565, 'only': 2495, 'than':

In [ ]:
# Reformating the dataset into csv for convenience 
def to_df (folder):
  data_dic = {}
  data_dic['doc'], data_dic['label'] = [], []
  for file in folder:
    fo = open(file)
    doc = fo.read()
    data_dic['doc'].append(doc)
    if 'pos' in file:
      data_dic['label'].append('pos')
    elif 'neg' in file:
      data_dic['label'].append('neg')
    else:
      print('error', file)
  df = pd.DataFrame.from_dict(data_dic)
  return df
    
data = to_df(pos_neg_paths)

data.head()

,doc,label
0,is evil dead ii a bad movie ? \nit's full of t...,pos
1,logical time travel movies are a near-impossib...,pos
2,robocop is an intelligent science fiction thri...,pos
3,historical epic as a genre was almost banished...,pos
4,"mike myers , you certainly did throw us a ? fr...",pos


In [ ]:
# Data preprocessing
stemmer = nltk.stem.porter.PorterStemmer()
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
def preprocessor (text):
  ## removing punctuations and characters
  text = re.sub(r'[^\w\s]', '', text)
  # stripping
  text = ' '.join([w.strip() for w in text.split()])
  # print(text)
  ## lowcasing
  text = text.lower()
  # ## removing stopword
  text = stopword_remover (text.split())
  # ##stemmming
  # text = [stemmer.stem(w) for w in text]
  # ## lematization
  # text = [lemmatizer.lemmatize(w) for w in text]
  return text

data['doc'] = data['doc'].apply(lambda x:  preprocessor (x) )
data 

,doc,label
0,"[evil, dead, ii, bad, movie, full, terrible, a...",pos
1,"[logical, time, travel, movies, nearimpossibil...",pos
2,"[robocop, intelligent, science, fiction, thril...",pos
3,"[historical, epic, genre, almost, banished, ho...",pos
4,"[mike, myers, certainly, throw, us, frickin, b...",pos
...,...,...
1995,"[slight, romantic, comedy, feminist, bent, one...",neg
1996,"[perhaps, best, remembered, recently, departed...",neg
1997,"[corruptor, big, silly, mess, action, movie, c...",neg
1998,"[heres, rarity, childrens, film, attempts, tac...",neg
